playlist_processing.ipynb

cleaned_song_meta.json(음악 메타 데이터), train.json(플레이리스트)를 사용해
플레이리스트에 존재하는 태그들을 song_id에 매핑한다.이 때 태그는 가장 많이 나온 10개의 태그만 저장한다.

count는 플레이리스트에 얼마나 많이 등장했는지 weight를 주기 위해 저장한다.

In [5]:
import json
import pandas as pd

"""
Method
"""

# Load cleaned song meta data(79,814)
def load_song_meta() -> pd.DataFrame:
    with open('./json_data/cleaned_song_meta.json', 'r', encoding='UTF8') as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    df = df[['id', 'song_name', 'artist_name_basket']]
    df['tags'] = [[] for _ in range(len(df))]
    df['count'] = 0
    return df

# Load playlist data(115,071)
def load_playlist() -> pd.DataFrame:
    with open('./json_data/train.json', 'r', encoding='UTF8') as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    df.sort_values(by=['like_cnt', 'updt_date'], ascending=False, inplace=True)
    return df

# Add playlist's @tags to each songs and update @count to give weights.
def processing_playlist(song_df: pd.DataFrame, playlist_df: pd.DataFrame) -> pd.DataFrame:
    for idx_, row in playlist_df.iterrows():
        cnt = row.like_cnt
        song_list = row.songs
        tags = row.tags
        for song in song_list:
            if song in song_df['id'].values:
                idx, = song_df.index[song_df['id'] == song].values
                song_df.at[idx, 'count'] += 1
                origin_tags = dict(song_df.at[idx, 'tags'])
                
                for tag in tags:
                    if tag not in origin_tags: origin_tags[tag] = 1
                    elif tag in origin_tags: origin_tags[tag] += 1
                
                if len(origin_tags) > 10:
                    new_dict = {}
                    for tag, cnt in sorted(list(origin_tags.items()), key=lambda x: -x[1])[:10]:
                        new_dict[tag] = cnt
                    song_df.at[idx, 'tags'] = list(new_dict.items())
                else:
                    song_df.at[idx, 'tags'] = list(origin_tags.items())
    
    for idx, row in song_df.iterrows():    
        song_df.at[idx, 'tags'] = list(dict(row.tags).keys())
    
    filter = song_df.tags.apply(lambda x: len(x) >= 10)
    song_df = song_df[filter]
    
    for idx, row in song_df.iterrows():
        song_df.at[idx, 'tags'] = ' '.join(song_df.at[idx, 'tags'])
    
    song_df = song_df[song_df['count'] >= 10]
        
    song_df.sort_values(by='count', ascending=False, inplace=True)
    return song_df
        

In [8]:
load_playlist()

,tags,id,plylst_title,songs,like_cnt,updt_date
46524,"[노동요, 취향저격, 드라이브, Pop, 기분전환, 트렌디, 스트레스, 신나는]",74220,HOT TRENDY POP: 놓쳐선 안될 'POP' (매주 업데이트),"[159317, 368264, 600999, 274905, 643023, 24070...",53211,2020-04-17 14:32:23.000
85071,"[감성, 새벽, 때껄룩, 잔잔한, 휴식, 팝송, 떼껄룩, 힐링, 팝, 신나는]",122388,❤️때껄룩님 ᴛᴀᴋᴇ ᴀ ʟᴏᴏᴋ 플레이리스트❤️,"[532771, 383775, 432753, 176384, 152694, 60126...",41844,2020-04-23 20:58:45.000
102651,"[매장음악, 카페, 러블리쏘, 분위기, 드라이브, 여행, 매력적인, 기분전환, 팝]",26174,듣다보면 '우와!' 하고 제목을 보게되는 팝,"[40160, 20039, 561488, 20051, 16242, 256568, 7...",27268,2020-04-19 21:22:11.000
101464,"[흥겨운, 드라이브, Pop, 기분전환, 경쾌한, 외출, 내적댄스, 기분업, 충전,...",72658,"약속 있어? 외출 전, 기분 UP 하고 싶을 때 들어봐! [매주 목요일]","[111779, 406878, 417725, 129878, 357770, 34879...",23965,2020-04-22 16:05:39.000
59894,"[힐링, 휴식, 기분전환]",113501,감성이 터지는 팝음악들,"[342524, 1233, 417666, 394523, 107455, 17206, ...",23732,2015-07-29 08:25:35.000
...,...,...,...,...,...,...
28916,[팝],78820,웃고 살자구요,"[401310, 13937, 668430, 384056, 87158, 145122,...",0,2005-03-22 03:20:20.000
52372,[팝],128239,내 핸펀 단골곡.,"[174974, 583613, 537068, 150525, 665627, 17121...",0,2005-02-28 12:13:04.000
20671,[발라드],37725,그렇게 너를 사랑해,"[293041, 657837, 597379, 408232, 145523, 21013...",0,2005-02-13 12:52:54.000
41106,"[설렘, 사랑]",72576,사랑해... 사랑해요,"[319693, 400275, 626919, 615275, 674884, 69135...",0,2005-01-16 16:36:35.000


In [6]:
processed_song = processing_playlist(load_song_meta(), load_playlist())

In [7]:
processed_song

,id,song_name,artist_name_basket,tags,count
16162,144663,밤편지,[아이유],감성 새벽 밤 인디 새벽감성 여름밤 잠들기전 밤에듣기좋은노래 새벽에듣기좋은노래 공부,2175
13048,116573,안아줘,[정준일],새벽 감성 밤 힐링 기분전환 가을 카페 잔잔 가을감성 노래추천,2121
40220,357367,비,[폴킴],발라드 비오는날 비 밤 혼자 장마 감성발라드 편안한 센치한 감성적인,1981
41312,366786,가끔 미치도록 네가 안고 싶어질 때가 있어,[가을방학],새벽 인디 밤 감성 새벽감성 잠들기전 여름밤 밤에듣기좋은노래 새벽에듣기좋은노래 공부,1919
14889,133143,"그대와 나, 설레임 (Feat. 소울맨)",[어쿠스틱 콜라보],기분전환 힐링 카페 발라드 새벽 밤 기분좋은 인디음악 잔잔 노래추천,1633
...,...,...,...,...,...
62958,558987,Blowin My Mind,[브라운 아이드 소울],브라운아이드소울 발라드 노래모음 브아솔 사랑노래 새벽에듣기좋은 2000년대 감성충만...,10
5682,50998,Gravity,[로이킴],감성 잔잔한 인디 겨울 밤 우울 새벽 밤에 숨은명곡 매장음악,10
5701,51129,Wax Mannequeen,[ELO],새벽 밤 잔잔한 감성 힙합 겨울 눈오는날 카페 눈 첫눈,10
75893,672349,DRAW (9월의 Hidden Track),[마틴스미스],인디 감성 감성적 느낌있는 인디음악 매력적인 인디뮤직 유니크 매력적 트렌디,10


In [29]:
processed_song.to_json('processed_song.json', orient='records')